In [12]:
import numpy as np
import sys
import math

In [23]:
#python p1_camera.py params.txt points.txt
np.set_printoptions(precision=1, suppress=True)
#np.set_printoptions(suppress=True)
PARAMS_FILE = "hw2_data_post/params_p1_v3.txt"#sys.argv[1]
POINTS_FILE = "hw2_data_post/points_p1_v3.txt"#sys.argv[2]

In [24]:
with open(PARAMS_FILE, mode='r') as params_fd:
    rotation_angles = np.array(params_fd.readline().strip("\n").split(), dtype='f')
    translation_vector = np.array(params_fd.readline().strip("\n").split(), dtype='f').reshape((3, 1))
    focal_length, pixel_d, vc, uc = np.array(params_fd.readline().strip("\n").split(), dtype='f')

In [25]:
#translation_vector, rotation_angles

In [26]:
rx_matrix = np.eye(3)
rx_matrix[1:3, 1:3] = np.array([[np.cos(rotation_angles[0] * (math.pi/180)), -1*np.sin(rotation_angles[0] * (math.pi/180))], [np.sin(rotation_angles[0] * (math.pi/180)), np.cos(rotation_angles[0] * (math.pi/180))]])
rz_matrix = np.eye(3)
rz_matrix[0:2, 0:2] = np.array([[np.cos(rotation_angles[2] * (math.pi/180)), -1*np.sin(rotation_angles[2] * (math.pi/180))], [np.sin(rotation_angles[2] * (math.pi/180)), np.cos(rotation_angles[2] * (math.pi/180))]])
ry_matrix = np.array([[np.cos(rotation_angles[1] * (math.pi/180)), 0, np.sin(rotation_angles[1] * (math.pi/180))], [0, 1, 0], [-1*np.sin(rotation_angles[1] * (math.pi/180)), 0, np.cos(rotation_angles[1] * (math.pi/180))]])
rotation_matrix = np.dot(np.dot(rx_matrix, ry_matrix), rz_matrix)

In [35]:
R_matrix = np.concatenate((rotation_matrix.T, np.dot(-1*rotation_matrix.T, translation_vector)), axis=1)

In [44]:
for i in range(len(rotation_matrix)):
    print(*np.around(rotation_matrix[i], decimals=1), sep=", ")

0.6, -0.8, 0.2
0.7, 0.7, 0.3
-0.3, -0.0, 1.0


In [37]:
sx, sy = focal_length/(pixel_d*0.001), focal_length/(pixel_d*0.001)
K_matrix =  np.array([[sx, 0, uc], 
                      [0, sy, vc],
                      [0, 0, 1]])

In [43]:
for i in range(len(K_matrix)):
    print(*np.around(K_matrix[i], decimals=1), sep=", ")

1000.0, 0.0, 1200.0
0.0, 1000.0, 800.0
0.0, 0.0, 1.0


In [42]:
M_matrix = np.dot(K_matrix, R_matrix)
for i in range(len(M_matrix)):
    print(*np.around(M_matrix[i], decimals=1), sep=", ")

841.4, 1016.9, 835.4, -60772.1
-615.5, 859.2, 723.1, -19050.6
0.2, 0.3, 1.0, -51.4


In [48]:
with open(POINTS_FILE, mode='r') as points:
    visible = "visible:"
    hidden = "hidden:"
    for i, point in enumerate(points):
        point = point.strip("\n").split()
        homogeneous_points = np.array(point + [1], dtype='f').reshape((4,1))
        image_coords = np.dot(M_matrix, homogeneous_points).flatten()
        #Z coordinate is still preserved
        if image_coords[2] > 0:
            visible += " {}".format(i)
        else:
            hidden += " {}".format(i)
        affine_coordinates = np.around((image_coords / image_coords[2])[0:2][::-1], decimals=1).tolist()
        #Bounding box: 4000 rows, 6000 columns
        if affine_coordinates[0] >= 0 and affine_coordinates[0] < 4000 and affine_coordinates[1] >= 0 and affine_coordinates[1] < 6000:
            inside = 'inside'
        else:
            inside = 'outside'
        print("{}: {} => {} {}".format(i, " ".join(map(str, map(float, point))), " ".join(map(str, affine_coordinates)), inside))
    print(visible + "\n" + hidden)

0: 100.0 15.0 90.0 => -47.5 2054.1 outside
1: -100.0 800.0 1500.0 => 1161.7 1230.3 inside
2: 10.0 -500.0 -500.0 => 1250.7 1499.1 inside
3: -30.0 10.0 20.0 => -641.4 1688.2 outside
visible: 0 1
hidden: 2 3
